In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data, wb
import datetime as dt
import matplotlib.pyplot as plt
import talib as ta
import finsymbols as sym
import operator
import pickle
import os.path

In [2]:
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 10)

Populating the interactive namespace from numpy and matplotlib


In [3]:
cannabis = ['ABBV', 'CARA', 'GWPH', 'INSY', 'ZDPY']
prisons = ['CXW', 'GEO']

In [4]:
cannabis2 = ['AERO','ABBV','ACAN','ATTBF','AXIM','BLPG','TECR','PRRE',
             'CANN','CVSI','CARA','CBDS','CBIS','CGRW','CHUM','CNAB','PKPH','CVSI',
             'FULL','GRCU','GWPH','HLIX','HLSPY','ICBU','IGC','IMLFF','INQD','INSY',
             'LXRP','MCIG','MJMJ','MJNA','MJNE','MNTR','MDCL','MJN','MYHI','NTRR','OGRMF',
             'OXIS','PLPL','PHOT','RSSFF','SMG','SRNA','TRTC','TWMJF','TURV',
             'XXII','ZDPY','ZYNE','IIPR']

In [5]:
def PriceCrossoverWithoutTradingCommisions(st, outputColumn, inputColumn):
    st[outputColumn] = ((st['Close'] / st['Close'].shift(1))-1)*st[inputColumn]
    st.ix[0, outputColumn] = 0

In [6]:
def PriceCrossoverWithTradingCommition(st, shiftedInputColumn, inputColumn, outputColumn, signalColumn, tempColumn):
    st[shiftedInputColumn] = st[inputColumn].shift(1)
    st[tempColumn] = st[signalColumn]
    sma5tc = 0
    for i, r in enumerate(st.iterrows()):
        if (r[1][signalColumn] == 1 or r[1][signalColumn] == -1) and r[1][inputColumn] != r[1][shiftedInputColumn]:
            sma5tc = 0.01
        else:
            sma5tc = 0.00
        st.ix[i, tempColumn] = sma5tc

    st[outputColumn] = (((st['Close']/st['Close'].shift(1))-1)-st[tempColumn])*st[inputColumn]
    st.ix[0, outputColumn] = 0

In [7]:
def CumulativeReturns(st, crt, crtc, bhcrt, drt, drtc, bhdrt, title):
    st[crt] = np.cumprod(st[drt]+1)-1
    st[crtc] = np.cumprod(st[drtc]+1)-1
    st[bhcrt] = np.cumprod(st[bhdrt]+1)-1

In [8]:
def StrategiesPerformanceMetrics(st, crt, crtc, bhcrt, drt, drtc, bhdrt, colTitle1, colTitle2, colTitle3):
    # Annualized Returns
    styrt = st.ix[-1, crt]
    styrtc = st.ix[-1, crtc]
    bhyrt = st.ix[-1, bhcrt]

    # Annualized Standard Deviation
    ststd = np.std(st[drt])*np.sqrt(252)
    ststdc = np.std(st[drtc])*np.sqrt(252)
    bhstd = np.std(st[bhdrt])*np.sqrt(252)

    # Annualized Sharpe Ratio
    stsr = styrt / ststd
    stsrc = styrtc / ststdc
    bhsr = bhyrt / bhstd

    # Summary Results Data Table
#     data = [{'0': '', '1': colTitle1, '2': colTitle2, '3': colTitle3},
#             {'0': 'Annualized Return', '1': styrt, '2': styrtc, '3': bhyrt},
#             {'0': 'Annualized Standard Deviation', '1': ststd, '2': ststdc, '3': bhstd},
#             {'0': 'Annualized Sharpe Ratio (Rf=0%)', '1': stsr, '2': stsrc, '3': bhsr}]
    
#     table = pd.DataFrame(data)
#     print(table)
    return stsr, stsrc, bhsr

In [9]:
def GenerateTradingStrategy(st, strategyColumn, signalColumn): # Own stock=1 , not own stock=0, no short-selling
    st[strategyColumn] = 1
    sma5str = 0
    for i, r in enumerate(st.iterrows()):
        if r[1][signalColumn] == 1:
            sma5str = 1
        elif r[1][signalColumn] == -1:
            sma5str = 0
        else:
            sma5str = st[strategyColumn][i-1]
        st.ix[i, strategyColumn] = sma5str


In [10]:
def process(symbol):
    start = dt.datetime(2012, 1, 1)
    stock = data.DataReader(symbol, 'google', start)

    stock['sma5'] = ta.SMA(stock['Close'].values, 5)
    stock['ema5'] = ta.EMA(stock['Close'].values, 5)
    stock['ema21'] = ta.EMA(stock['Close'].values, 21)

    # BBANDS(20,2)
    stock['upper'], stock['middle'], stock['lower'] = ta.BBANDS(stock['Close'].values,
                                                                timeperiod=20,
                                                                nbdevup=2,
                                                                nbdevdn=2,
                                                                matype=0)
    
    # Parabolic Stop and Reverse SAR(0.02,0.2) (Acceleration Factor Increment, Maximum Acceleration Factor)
    stock['sar'] = ta.SAR(stock['High'].values, stock['Low'].values, acceleration=0.02, maximum=0.2)
    
    # Average Directional Moving Index, ADX(14)
    stock['adx'] = ta.ADX(stock['High'].values, stock['Low'].values, stock['Close'].values, timeperiod=14)
    stock['+di'] = ta.PLUS_DI(stock['High'].values, stock['Low'].values, stock['Close'].values, timeperiod=14)
    stock['-di'] = ta.MINUS_DI(stock['High'].values, stock['Low'].values, stock['Close'].values, timeperiod=14)
    
    # Commodity Channel Index (20,0.015)
    # +100 overbought below -100 oversold
    # Warn of extreme conditions
    # It is high when prices is too much above the average and too low when prices too below their average
    stock['cci'] = ta.CCI(stock['High'].values, stock['Low'].values, stock['Close'].values, timeperiod=20)
    
    # MACD - Moving Averages Convergence Divergence(12,26,9)
    # Strength of a trend by comparing two moving averages
    # When the signal line cross overs the MCAD line it is a trading signal
    # http://www.tadoc.org/indicator/MACD.htm
    # macd, macdsignal, macdhist = MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
    # http://www.investopedia.com/articles/technical/082701.asp
    stock['macd'], stock['macdema'], stock['macdhist'] = ta.MACD(stock['Close'].values,
                                                          fastperiod=12, slowperiod=26, signalperiod=9)
    # Rate of Change - ROC(21)
    # As a momentum oscillator, ROC signals include centerline crossovers, divergences and overbought-oversold readings.
    # Prices are rising as long as the Rate-of-Change remains positive.
    # Prices are falling when the Rate-of-Change is negative. 
    
    stock['roc'] = ta.ROC(stock['Close'].values, timeperiod=21)
    
    # Relative Strength Index = RSI(14)
    # A momentum oscillator that measures the speed and change of price movements. 
    # Between 0 and 100
    # Overbought when above 70 and oversold when below 30
    # http://www.tadoc.org/indicator/RSI.htm
    # http://www.investopedia.com/articles/technical/03/070203.asp
    
    stock['rsi'] = ta.RSI(stock['Close'].values, timeperiod=14)
    
    # Stochastic Oscillator Full - STO(14,3,3)
    # A momentum indicator
    # Changes direction before the price
    # slowk, slowd = STOCH(high, low, close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    # http://www.tadoc.org/indicator/STOCH.htm
    # https://www.metastock.com/customer/resources/taaz/?p=106
    stock['slowk'], stock['slowd'] = ta.STOCH(stock['High'].values, stock['Low'].values, stock['Close'].values,
                                         fastk_period=14, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    # Williams %R(14)
    # Inverse of fast stochastic oscillator
    # Fast stochastic oscillator and Willaim %R is the same scaling is different
    stock['wpr'] = ta.WILLR(stock['High'].values, stock['Low'].values, stock['Close'].values, timeperiod=14)

    stock['Close(-1)'] = stock['Close'].shift(1)
    stock['Close(-2)'] = stock['Close'].shift(2)

    stock['bhdrt'] = (stock['Close']/stock['Close(-1)'])-1
    stock.ix[0, 'bhdrt'] = 0

    # Cumulative Daily Return
    stock['bhcrt'] = np.cumprod(stock['bhdrt']+1)-1

    # Previous Periods Data (avoid backtesting bias)
    stock['sma5(-1)'] = stock['sma5'].shift(1)
    stock['sma5(-2)'] = stock['sma5'].shift(2)
    stock['ema5(-1)'] = stock['ema5'].shift(1)
    stock['ema21(-1)'] = stock['ema21'].shift(1)
    stock['ema5(-2)'] = stock['ema5'].shift(2)
    stock['ema21(-2)'] = stock['ema21'].shift(2)
    stock['lower(-1)'] = stock['lower'].shift(1)
    stock['upper(-1)'] = stock['upper'].shift(1)
    stock['lower(-2)'] = stock['lower'].shift(2)
    stock['upper(-2)'] = stock['upper'].shift(2)
    stock['sar(-1)'] = stock['sar'].shift(1)
    stock['sar(-2)'] = stock['sar'].shift(2)
    stock['adx(-1)'] = stock['adx'].shift(1)
    stock['+di(-1)'] = stock['+di'].shift(1)
    stock['-di(-1)'] = stock['-di'].shift(1)
    stock['+di(-2)'] = stock['+di'].shift(2)
    stock['-di(-2)'] = stock['-di'].shift(2)
    stock['cci(-1)'] = stock['cci'].shift(1)
    stock['cci(-2)'] = stock['cci'].shift(2)
    stock['macd(-1)'] = stock['macd'].shift(1)
    stock['macdema(-1)'] = stock['macdema'].shift(1)
    stock['macd(-2)'] = stock['macd'].shift(2)
    stock['macdema(-2)'] = stock['macdema'].shift(2)
    stock['roc(-1)'] = stock['roc'].shift(1)
    stock['roc(-2)'] = stock['roc'].shift(2)
    stock['rsi(-1)'] = stock['rsi'].shift(1)
    stock['rsi(-2)'] = stock['rsi'].shift(2)
    stock['slowd(-1)'] = stock['slowd'].shift(1)
    stock['slowd(-2)'] = stock['slowd'].shift(2)
    stock['wpr(-1)'] = stock['wpr'].shift(1)
    stock['wpr(-2)'] = stock['wpr'].shift(2)

    stock['sma5sig'] = 0
    sma5sig = 0
    for i, r in enumerate(stock.iterrows()):
        if r[1]['Close(-2)'] < r[1]['sma5(-2)'] and r[1]['Close(-1)'] > r[1]['sma5(-1)']:
            sma5sig = 1.0
        elif r[1]['Close(-2)'] > r[1]['sma5(-2)'] and r[1]['Close(-1)'] < r[1]['sma5(-1)']:
            sma5sig = -1.0
        else:
            sma5sig = 0.0
        stock.ix[i, 'sma5sig'] = sma5sig

    GenerateTradingStrategy(stock, 'sma5str', 'sma5sig')

    # Generate Trading Signals (buy=1 , sell=-1, do nothing=0)
    stock['emasig'] = 0
    emasig = 0
    for i, r in enumerate(stock.iterrows()):
        if r[1]['ema5(-2)'] < r[1]['ema21(-2)'] and r[1]['ema5(-1)'] > r[1]['ema21(-1)']:
            emasig = 1
        elif r[1]['ema5(-2)'] > r[1]['ema21(-2)'] and r[1]['ema5(-1)'] < r[1]['ema21(-1)']:
            emasig = -1
        else:
            emasig = 0
        stock.ix[i, 'emasig'] = emasig

    GenerateTradingStrategy(stock, 'emastr', 'emasig')

    stock['bbsig'] = 0
    bbsig = 0
    for i, r in enumerate(stock.iterrows()):
        if r[1]['Close(-2)'] < r[1]['lower(-2)'] and r[1]['Close(-1)'] > r[1]['lower(-1)']:
            bbsig = 1
        elif r[1]['Close(-2)'] < r[1]['upper(-2)'] and r[1]['Close(-1)'] > r[1]['upper(-1)']:
            bbsig = -1
        else:
            bbsig = 0
        stock.ix[i, 'bbsig'] = bbsig

    GenerateTradingStrategy(stock, 'bbstr', 'bbsig')

    # SAR Trading Signals
    stock['sarsig'] = 0
    sarsig = 0
    for i, r in enumerate(stock.iterrows()):
        if r[1]['Close(-2)'] < r[1]['sar(-2)'] and r[1]['Close(-1)'] > r[1]['sar(-1)']:
            sarsig = 1
        elif r[1]['Close(-2)'] > r[1]['sar(-2)'] and r[1]['Close(-1)'] < r[1]['sar(-1)']:
            sarsig = -1
        else:
            sarsig = 0
        stock.ix[i, 'sarsig'] = sarsig

    # Average Directional Movement Trade Signals
    stock['adxsig'] = 0
    adxsig = 0
    for i, r in enumerate(stock.iterrows()):
        if r[1]['+di(-2)'] < r[1]['-di(-2)'] and r[1]['+di(-1)'] > r[1]['-di(-1)'] and r[1]['adx(-1)'] > 20:
            adxsig = 1
        elif r[1]['+di(-2)'] > r[1]['-di(-2)'] and r[1]['+di(-1)'] < r[1]['-di(-1)'] and r[1]['adx(-1)'] > 20:
            adxsig = -1
        else:
            adxsig = 0
        stock.ix[i, 'adxsig'] = adxsig

    # TS (buy=1 , sell=-1, do nothing=0)
    stock['ccisig'] = 0
    ccisig = 0
    for i, r in enumerate(stock.iterrows()):
        if r[1]['cci(-2)'] < -100 and r[1]['cci(-1)'] > -100:
            ccisig = 1
        elif r[1]['cci(-2)'] < 100 and r[1]['cci(-1)'] > 100:
            ccisig = -1
        else:
            ccisig = 0
        stock.ix[i, 'ccisig'] = ccisig

    # MACD
    stock['macdsig'] = 0
    macdsig = 0
    for i, r in enumerate(stock.iterrows()):
        if r[1]['macd(-2)'] < r[1]['macdema(-2)'] and r[1]['macd(-1)'] > r[1]['macdema(-1)']:
            macdsig = 1
        elif r[1]['macd(-2)'] > r[1]['macdema(-2)'] and r[1]['macd(-1)'] < r[1]['macdema(-1)']:
            macdsig = -1
        else:
            macdsig = 0
        stock.ix[i, 'macdsig'] = macdsig

    # ROC
    stock['rocsig'] = 0
    rocsig = 0
    for i, r in enumerate(stock.iterrows()):
        if r[1]['roc(-2)'] < -10 and r[1]['roc(-1)'] > -10:
            rocsig = 1
        elif r[1]['roc(-2)'] < 10 and r[1]['roc(-1)'] > 10:
            rocsig = -1
        else:
            rocsig = 0
        stock.ix[i, 'rocsig'] = rocsig

    # Relative Strength Trading Signal
    stock['rsisig'] = 0
    rsisig = 0
    for i, r in enumerate(stock.iterrows()):
        if r[1]['rsi(-2)'] < 30 and r[1]['rsi(-1)'] > 30:
            rsisig = 1
        elif r[1]['rsi(-2)'] < 70 and r[1]['rsi(-1)'] > 70:
            rsisig = -1
        else:
            rsisig = 0
        stock.ix[i, 'rsisig'] = rsisig

    # Stochastic Oscillator Trading Signal
    stock['stosig'] = 0
    stosig = 0
    for i, r in enumerate(stock.iterrows()):
        if r[1]['slowd(-2)'] < 20 and r[1]['slowd(-1)'] > 20:
            stosig = 1
        elif r[1]['slowd(-2)'] < 80 and r[1]['slowd(-1)'] > 80:
            stosig = -1
        else:
            stosig = 0
        stock.ix[i, 'stosig'] = stosig

    # William %R Trading Signal
    stock['wprsig'] = 0
    wprsig = 0
    for i, r in enumerate(stock.iterrows()):
        if r[1]['wpr(-2)'] < -80 and r[1]['wpr(-1)'] > -80:
            wprsig = 1
        elif r[1]['wpr(-2)'] < -20 and r[1]['wpr(-1)'] > -20:
            wprsig = -1
        else:
            wprsig = 0
        stock.ix[i, 'wprsig'] = wprsig

    # SMA(5) and CCI(20,0.015)
    stock['ccismasig'] = 0
    ccismasig = 0
    for i, r in enumerate(stock.iterrows()):
        if r[1]['Close(-2)'] < r[1]['sma5(-2)'] and r[1]['Close(-1)'] > r[1]['sma5(-1)'] and r[1]['cci(-1)'] < -100:
            ccismasig = 1
        elif r[1]['Close(-2)'] > r[1]['sma5(-2)'] and r[1]['Close(-1)'] < r[1]['sma5(-1)'] and r[1]['cci(-1)'] > 100:
            ccismasig = -1
        else:
            ccismasig = 0
        stock.ix[i, 'ccismasig'] = ccismasig

    # SMA(5) and ROC(21)
    stock['rocsmasig'] = 0
    rocsmasig = 0
    for i, r in enumerate(stock.iterrows()):
        if r[1]['Close(-2)'] < r[1]['sma5(-2)'] and r[1]['Close(-1)'] > r[1]['sma5(-1)'] and r[1]['roc(-1)'] < -10:
            rocsmasig = 1
        elif r[1]['Close(-2)'] > r[1]['sma5(-2)'] and r[1]['Close(-1)'] < r[1]['sma5(-1)'] and r[1]['roc(-1)'] > 10:
            rocsmasig = -1
        else:
            rocsmasig = 0
        stock.ix[i, 'rocsmasig'] = rocsmasig

    # SMA(5) and STO(14,3,3)
    stock['stosmasig'] = 0
    stosmasig = 0
    for i, r in enumerate(stock.iterrows()):
        if r[1]['Close(-2)'] < r[1]['sma5(-2)'] and r[1]['Close(-1)'] > r[1]['sma5(-1)'] and r[1]['slowd(-1)'] < 20:
            stosmasig = 1
        elif r[1]['Close(-2)'] > r[1]['sma5(-2)'] and r[1]['Close(-1)'] < r[1]['sma5(-1)'] and r[1]['slowd(-1)'] > 80:
            stosmasig = -1
        else:
            stosmasig = 0
        stock.ix[i, 'stosmasig'] = stosmasig

    # SMA(5) and Williams %R(14)
    stock['wprsmasig'] = 0
    wprsmasig = 0
    for i, r in enumerate(stock.iterrows()):
        if r[1]['Close(-2)'] < r[1]['sma5(-2)'] and r[1]['Close(-1)'] > r[1]['sma5(-1)'] and r[1]['wpr(-1)'] < -80:
            wprsmasig = 1
        elif r[1]['Close(-2)'] > r[1]['sma5(-2)'] and r[1]['Close(-1)'] < r[1]['sma5(-1)'] and r[1]['wpr(-1)'] > -20:
            wprsmasig = -1
        else:
            wprsmasig = 0
        stock.ix[i, 'wprsmasig'] = wprsmasig    

    srDict = {}
    srcDict = {}

    GenerateTradingStrategy(stock, 'sma5str', 'sma5sig')
    PriceCrossoverWithoutTradingCommisions(stock, 'sma5drt', 'sma5str')
    PriceCrossoverWithTradingCommition(stock, 'sma5str(-1)', 'sma5str', 'sma5drtc', 'sma5sig', 'sma5tc')
    CumulativeReturns(stock, 'sma5crt', 'sma5crtc', 'bhcrt', 'sma5drt', 'sma5drtc', 'bhdrt', 'Simple Moving Average SMA(5) vs Buy & Hold')
    sma5sr, sma5src, sma5bhsr = StrategiesPerformanceMetrics(stock, 'sma5crt', 'sma5crtc', 'bhcrt', 'sma5drt', 'sma5drtc', 'bhdrt', 'SMA(5)', 'SMA(5)TC', 'B&H')
    srDict['sma5'] = sma5sr
    srcDict['sma5'] = sma5src
    
    GenerateTradingStrategy(stock, 'emastr', 'emasig')
    PriceCrossoverWithoutTradingCommisions(stock, 'emadrt', 'emastr')
    PriceCrossoverWithTradingCommition(stock, 'emastr(-1)', 'emastr', 'emadrtc', 'emasig', 'ematc')
    CumulativeReturns(stock, 'emacrt', 'emacrtc', 'bhcrt', 'emadrt', 'emadrtc', 'bhdrt', 'Exponential Moving Averages EMA(5 & 21) vs Buy & Hold')
    emasr, emasrc, emabhsr = StrategiesPerformanceMetrics(stock, 'emacrt', 'emacrtc', 'bhcrt', 'emadrt', 'emadrtc', 'bhdrt', 'EMA(5 & 21)', 'EMA(5 & 21)TC', 'B&H')
    srDict['ema'] = emasr
    srcDict['ema'] = emasrc

    GenerateTradingStrategy(stock, 'bbstr', 'bbsig')
    PriceCrossoverWithoutTradingCommisions(stock, 'bbdrt', 'bbstr')
    PriceCrossoverWithTradingCommition(stock, 'bbstr(-1)', 'bbstr', 'bbdrtc', 'bbsig', 'bbtc')
    CumulativeReturns(stock, 'bbcrt', 'bbcrtc', 'bhcrt', 'bbdrt', 'bbdrtc', 'bhdrt', 'Bollinger Bands BB(20,2) vs Buy & Hold')
    bbsr, bbsrc, bbbhsr = StrategiesPerformanceMetrics(stock, 'bbcrt', 'bbcrtc', 'bhcrt', 'bbdrt', 'bbdrtc', 'bhdrt', 'BB(20,2)', 'BB(20,2)TC', 'B&H')
    srDict['bb'] = bbsr
    srcDict['bb'] = bbsrc

    GenerateTradingStrategy(stock, 'sarstr', 'sarsig')
    PriceCrossoverWithoutTradingCommisions(stock, 'sardrt', 'sarstr')
    PriceCrossoverWithTradingCommition(stock, 'sarstr(-1)', 'sarstr', 'sardrtc', 'sarsig', 'sartc')
    CumulativeReturns(stock, 'sarcrt', 'sarcrtc', 'bhcrt', 'sardrt', 'sardrtc', 'bhdrt', 'Parabolic Stop and Reverse SAR(0.02,0.2) vs Buy & Hold')
    sarsr, sarsrc, sarbhsr = StrategiesPerformanceMetrics(stock, 'sarcrt', 'sarcrtc', 'bhcrt', 'sardrt', 'sardrtc', 'bhdrt', 'SAR(0.02,0.2)', 'SAR(0.02,0.2)TC', 'B&H')
    srDict['sar'] = sarsr
    srcDict['sar'] = sarsrc

    GenerateTradingStrategy(stock, 'adxstr', 'adxsig')
    PriceCrossoverWithoutTradingCommisions(stock, 'adxdrt', 'adxstr')
    PriceCrossoverWithTradingCommition(stock, 'adxstr(-1)', 'adxstr', 'adxdrtc', 'adxsig', 'adxtc')
    CumulativeReturns(stock, 'adxcrt', 'adxcrtc', 'bhcrt', 'adxdrt', 'adxdrtc', 'bhdrt', 'Average Directional Movement Index ADX(14) vs Buy & Hold')
    adxsr, adxsrc, adxbhsr = StrategiesPerformanceMetrics(stock, 'adxcrt', 'adxcrtc', 'bhcrt', 'adxdrt', 'adxdrtc', 'bhdrt', 'ADX(14)', 'ADX(14)TC', 'B&H')
    srDict['adx'] = adxsr
    srcDict['adx'] = adxsrc

    GenerateTradingStrategy(stock, 'ccistr', 'ccisig')
    PriceCrossoverWithoutTradingCommisions(stock, 'ccidrt', 'ccistr')
    PriceCrossoverWithTradingCommition(stock, 'ccistr(-1)', 'ccistr', 'ccidrtc', 'ccisig', 'ccitc')
    CumulativeReturns(stock, 'ccicrt', 'ccicrtc', 'bhcrt', 'ccidrt', 'ccidrtc', 'bhdrt', 'Commodity Channel Index CCI(20,0.015) vs Buy & Hold')
    ccisr, ccisrc, ccibhsr = StrategiesPerformanceMetrics(stock, 'ccicrt', 'ccicrtc', 'bhcrt', 'ccidrt', 'ccidrtc', 'bhdrt', 'CCI(20,0.015)', 'CCI(20,0.015)TC', 'B&H')
    srDict['cci'] = ccisr
    srcDict['cci'] = ccisrc

    GenerateTradingStrategy(stock, 'macdstr', 'macdsig')
    PriceCrossoverWithoutTradingCommisions(stock, 'macddrt', 'macdstr')
    PriceCrossoverWithTradingCommition(stock, 'macdstr(-1)', 'macdstr', 'macddrtc', 'macdsig', 'macdtc')
    CumulativeReturns(stock, 'macdcrt', 'macdcrtc', 'bhcrt', 'macddrt', 'macddrtc', 'bhdrt', 'Moving Averages Convergence Divergence MACD(12,26,9) vs Buy & Hold')
    macdsr, macdsrc, macdbhsr = StrategiesPerformanceMetrics(stock, 'macdcrt', 'macdcrtc', 'bhcrt', 'macddrt', 'macddrtc', 'bhdrt', 'MACD(12,26,9)', 'MACD(12,26,9)TC', 'B&H')
    srDict['macd'] = macdsr
    srcDict['macd'] = macdsrc

    GenerateTradingStrategy(stock, 'rocstr', 'rocsig')
    PriceCrossoverWithoutTradingCommisions(stock, 'rocdrt', 'rocstr')
    PriceCrossoverWithTradingCommition(stock, 'rocstr(-1)', 'rocstr', 'rocdrtc', 'rocsig', 'roctc')
    CumulativeReturns(stock, 'roccrt', 'roccrtc', 'bhcrt', 'rocdrt', 'rocdrtc', 'bhdrt', 'Rate of Change ROC(21) vs Buy & Hold')
    rocsr, rocsrc, rocbhsr = StrategiesPerformanceMetrics(stock, 'sma5crt', 'sma5crtc', 'bhcrt', 'sma5drt', 'sma5drtc', 'bhdrt', 'ROC(21)', 'ROC(21)TC', 'B&H')
    srDict['roc'] = rocsr
    srcDict['roc'] = rocsrc

    GenerateTradingStrategy(stock, 'rsistr', 'rsisig')
    PriceCrossoverWithoutTradingCommisions(stock, 'rsidrt', 'rsistr')
    PriceCrossoverWithTradingCommition(stock, 'rsistr(-1)', 'rsistr', 'rsidrtc', 'rsisig', 'rsitc')
    CumulativeReturns(stock, 'rsicrt', 'rsicrtc', 'bhcrt', 'rsidrt', 'rsidrtc', 'bhdrt', 'Relative Strength Index RSI(14) vs Buy & Hold')
    rsisr, rsisrc, rsibhsr = StrategiesPerformanceMetrics(stock, 'rsicrt', 'rsicrtc', 'bhcrt', 'rsidrt', 'rsidrtc', 'bhdrt', 'RSI(14)', 'RSI(14)TC', 'B&H')
    srDict['rsi'] = rsisr
    srcDict['rsi'] = rsisrc

    GenerateTradingStrategy(stock, 'stostr', 'stosig')
    PriceCrossoverWithoutTradingCommisions(stock, 'stodrt', 'stostr')
    PriceCrossoverWithTradingCommition(stock, 'stostr(-1)', 'stostr', 'stodrtc', 'stosig', 'stotc')
    CumulativeReturns(stock, 'stocrt', 'stocrtc', 'bhcrt', 'stodrt', 'stodrtc', 'bhdrt', 'Stochastic Oscillator Full STO(14,3,3) vs Buy & Hold')
    stosr, stosrc, stobhsr = StrategiesPerformanceMetrics(stock, 'stocrt', 'stocrtc', 'bhcrt', 'stodrt', 'stodrtc', 'bhdrt', 'STO(14,3,3)', 'STO(14,3,3)TC', 'B&H')
    srDict['sto'] = stosr
    srcDict['sto'] = stosrc

    GenerateTradingStrategy(stock, 'wprstr', 'wprsig')
    PriceCrossoverWithoutTradingCommisions(stock, 'wprdrt', 'wprstr')
    PriceCrossoverWithTradingCommition(stock, 'wprstr(-1)', 'wprstr', 'wprdrtc', 'wprsig', 'wprtc')
    CumulativeReturns(stock, 'wprcrt', 'wprcrtc', 'bhcrt', 'wprdrt', 'wprdrtc', 'bhdrt', 'Williams %R(14) vs Buy & Hold')
    wprsr, wprsrc, wprbhsr = StrategiesPerformanceMetrics(stock, 'wprcrt', 'wprcrtc', 'bhcrt', 'wprdrt', 'wprdrtc', 'bhdrt', '%R(14)', '%R(14)TC', 'B&H')
    srDict['wpr'] = wprsr
    srcDict['wpr'] = wprsrc

    GenerateTradingStrategy(stock, 'ccismastr', 'ccismasig')
    PriceCrossoverWithoutTradingCommisions(stock, 'ccismadrt', 'ccismastr')
    PriceCrossoverWithTradingCommition(stock, 'ccismastr(-1)', 'ccismastr', 'ccismadrtc', 'ccismasig', 'ccismatc')
    CumulativeReturns(stock, 'ccismacrt', 'ccismacrtc', 'bhcrt', 'ccismadrt', 'ccismadrtc', 'bhdrt', 'Simple Moving Average SMA(5) & Commodity Channel Index CCI(20,0.015) vs Buy & Hold')
    ccismasr, ccismasrc, ccismabhsr = StrategiesPerformanceMetrics(stock, 'ccismacrt', 'ccismacrtc', 'bhcrt', 'ccismadrt', 'ccismadrtc', 'bhdrt', 'CCI(20,0.015)', 'CCI(20,0.015)TC', 'B&H')
    srDict['ccisma'] = ccismasr
    srcDict['ccisma'] = ccismasrc

    GenerateTradingStrategy(stock, 'rocsmastr', 'rocsmasig')
    PriceCrossoverWithoutTradingCommisions(stock, 'rocsmadrt', 'rocsmastr')
    PriceCrossoverWithTradingCommition(stock, 'rocsmastr(-1)', 'rocsmastr', 'rocsmadrtc', 'rocsmasig', 'rocsmatc')
    CumulativeReturns(stock, 'rocsmacrt', 'rocsmacrtc', 'bhcrt', 'rocsmadrt', 'rocsmadrtc', 'bhdrt', 'Simple Moving Average SMA(5) & Rate of Change ROC(21) vs Buy & Hold')
    rocsmasr, rocsmasrc, rocsmabhsr = StrategiesPerformanceMetrics(stock, 'rocsmacrt', 'rocsmacrtc', 'bhcrt', 'rocsmadrt', 'rocsmadrtc', 'bhdrt', '---', '---TC', 'B&H')
    srDict['rocsma'] = rocsmasr
    srcDict['rocsma'] = rocsmasrc

    GenerateTradingStrategy(stock, 'stosmastr', 'stosmasig')
    PriceCrossoverWithoutTradingCommisions(stock, 'stosmadrt', 'stosmastr')
    PriceCrossoverWithTradingCommition(stock, 'stosmastr(-1)', 'stosmastr', 'stosmadrtc', 'stosmasig', 'stosmatc')
    CumulativeReturns(stock, 'stosmacrt', 'stosmacrtc', 'bhcrt', 'stosmadrt', 'stosmadrtc', 'bhdrt', 'Simple Moving Average SMA(5) & STO buy and hold')
    stosmasr, stosmasrc, stosmabhsr = StrategiesPerformanceMetrics(stock, 'stosmacrt', 'stosmacrtc', 'bhcrt', 'stosmadrt', 'stosmadrtc', 'bhdrt', '---', '---TC', 'B&H')
    srDict['stosma'] = stosmasr
    srcDict['stosma'] = stosmasrc

    GenerateTradingStrategy(stock, 'wprsmastr', 'wprsmasig')
    PriceCrossoverWithoutTradingCommisions(stock, 'wprsmadrt', 'wprsmastr')
    PriceCrossoverWithTradingCommition(stock, 'wprsmastr(-1)', 'wprsmastr', 'wprsmadrtc', 'wprsmasig', 'wprsmatc')
    CumulativeReturns(stock, 'wprsmacrt', 'wprsmacrtc', 'bhcrt', 'wprsmadrt', 'wprsmadrtc', 'bhdrt', 'Simple Moving Average SMA(5) & William %R(14) vs Buy & Hold')
    wprsmasr, wprsmasrc, wprsmabhsr = StrategiesPerformanceMetrics(stock, 'wprsmacrt', 'wprsmacrtc', 'bhcrt', 'wprsmadrt', 'wprsmadrtc', 'bhdrt', '---', '---TC', 'B&H')
    srDict['wprsma'] = wprsmasr
    srcDict['wprsma'] = wprsmasrc
    
    bhVal = wprsmabhsr
    srDict_filtered = {k:v for k,v in srDict.iteritems() if v > bhVal}
    srcDict_filtered = {k:v for k,v in srcDict.iteritems() if v > bhVal}
    srDict_filtered_sorted = sorted(srDict_filtered.iteritems(), key=operator.itemgetter(1), reverse=True)
    srcDict_filtered_sorted = sorted(srcDict_filtered.iteritems(), key=operator.itemgetter(1), reverse=True)
    print bhVal
#     print srDict_sorted
#     print srcDict_sorted
#     print srDict_filtered_sorted
#     print srcDict_filtered_sorted
    return fbhVal, srDict_filtered_sorted, srcDict_filtered_sorted

In [11]:
# process('NOW')

In [12]:
for can in cannabis:
    print can
#     process(can)
    print "==========================="

ABBV
CARA
GWPH
INSY
ZDPY


{'sector': u'Industrials', 'industry': u'Industrial Conglomerates', 'symbol': u'MMM', 'company': u'3M Company', 'headquarters': u'St. Paul, Minnesota'}

In [13]:
def processStockList(symbols, pickleJar):

    if os.path.isfile(pickleJar):
        with open(pickleJar, 'rb') as outfile:
            result = pickle.load(outfile)
    else:
        result = {}
    for can in symbols:
        if can in result.keys():
            print can, "Already Processed"
            continue
        print can
        try:
            result[can] = process(can)
        except:
            print "Error on", can
        print "==========================="

    with open(pickleJar, 'wb') as outfile:
        pickle.dump(result, outfile)

In [14]:
sp500 = sym.get_sp500_symbols()
sp500_symbols = [item['symbol'] for item in sp500]

processStockList(sp500_symbols, "sp500.pkl")

MMM Already Processed
ABT Already Processed
ABBV Already Processed
ACN Already Processed
ATVI Already Processed
AYI Already Processed
ADBE Already Processed
AMD Already Processed
AAP Already Processed
AES Already Processed
AET Already Processed
AMG Already Processed
AFL Already Processed
A Already Processed
APD Already Processed
AKAM Already Processed
ALK Already Processed
ALB
-0.874396542666
Error on ALB
ARE
Error on ARE
ALXN
2.54900199937
Error on ALXN
ALGN Already Processed
ALLE Already Processed
AGN Already Processed
ADS Already Processed
LNT Already Processed
ALL Already Processed
GOOGL Already Processed
GOOG Already Processed
MO Already Processed
AMZN Already Processed
AEE Already Processed
AAL Already Processed
AEP Already Processed
AXP Already Processed
AIG Already Processed
AMT
Error on AMT
AWK Already Processed
AMP Already Processed
ABC Already Processed
AME Already Processed
AMGN Already Processed
APH Already Processed
APC Already Processed
ADI Already Processed
ANSS Already

In [15]:
pickleJar = "cannabis2.pkl"

with open(pickleJar, 'rb') as outfile:
    result = pickle.load(outfile)
    
    for key, value in result.iteritems():
        print value
        break

(0.00043665671719453418, [('rocsma', 2.0113869377989428), ('sma5', 1.8960484901917525), ('roc', 1.8960484901917525), ('sar', 1.4108935289319935), ('ema', 1.0709466571822923), ('cci', 1.0450475393410761), ('rsi', 0.64504129562036161), ('stosma', 0.49395628881639242), ('adx', 0.27178189901749406), ('macd', 0.2635292452736771), ('bb', 0.076479840081344383), ('sto', 0.027312059790970965)], [('rocsma', 1.8505498138740157), ('sar', 1.0511804792790684), ('ema', 0.89812270308828124), ('cci', 0.83956524895473872), ('sma5', 0.70832859475691134), ('roc', 0.70832859475691134), ('rsi', 0.63167775152912886), ('stosma', 0.42335130685176081), ('adx', 0.2104360743328553), ('macd', 0.1211983813413739), ('bb', 0.0040236153083142764)])


In [16]:
processStockList(cannabis2, "cannabis2.pkl")

AERO Already Processed
ABBV Already Processed
ACAN
Error on ACAN
ATTBF
Error on ATTBF
AXIM
Error on AXIM
BLPG
Error on BLPG
TECR
Error on TECR
PRRE
Error on PRRE
CANN
Error on CANN
CVSI
Error on CVSI
CARA Already Processed
CBDS
Error on CBDS
CBIS
Error on CBIS
CGRW
Error on CGRW
CHUM
Error on CHUM
CNAB
Error on CNAB
PKPH
Error on PKPH
CVSI
Error on CVSI
FULL Already Processed
GRCU
Error on GRCU
GWPH Already Processed
HLIX
Error on HLIX
HLSPY
Error on HLSPY
ICBU Already Processed
IGC Already Processed
IMLFF
Error on IMLFF
INQD
Error on INQD
INSY Already Processed
LXRP
Error on LXRP
MCIG
Error on MCIG
MJMJ
Error on MJMJ
MJNA
Error on MJNA
MJNE
Error on MJNE
MNTR
Error on MNTR
MDCL
Error on MDCL
MJN Already Processed
MYHI
Error on MYHI
NTRR
Error on NTRR
OGRMF
Error on OGRMF
OXIS
Error on OXIS
PLPL
Error on PLPL
PHOT
Error on PHOT
RSSFF
Error on RSSFF
SMG Already Processed
SRNA
Error on SRNA
TRTC
Error on TRTC
TWMJF
Error on TWMJF
TURV
Error on TURV
XXII
Error on XXII
ZDPY
Error on ZDPY
Z

In [27]:
pickleJar = "sp500.pkl"
with open(pickleJar, 'rb') as outfile:
    result = pickle.load(outfile)
    max_item1 = 0
    max_item2 = 0
    ret_item1 = []
    ret_item2 = []
    max_sym = "NOSYM"
    ret_hs = 0
    for key, value in result.iteritems():
        hs, item1, item2 = value
        if item1:
            v1, v2 = item1[0]
            if (v2 / hs) > max_item1:
                max_item1 = v2 / hs
                max_sym = key
                ret_item1 = item1
                ret_item2 = item2
                ret_hs = hs
    print ret_hs, max_sym, max_item1, ret_item1, ret_item2

 0.00374349588964 FAST 859.788758152 [('bb', 3.2186156821044074), ('rsi', 2.9581604609094225), ('sto', 2.1411847139683311), ('rocsma', 1.7032806610449889), ('cci', 1.5950769178659256), ('stosma', 1.5206873976758879), ('sar', 0.74937705973286295), ('wpr', 0.4203976892939757), ('ccisma', 0.29141653796585704), ('wprsma', 0.25858014473156871)] [('rsi', 2.4656129639309969), ('bb', 1.5995997307935115), ('rocsma', 1.4198095694669666), ('sto', 0.61270783311388444), ('stosma', 0.25245845497214481)]
